In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import string
!pip install -U scikit-learn scipy matplotlib

!pip install textblob
!pip install profanity
!pip install contractions
!pip install autocorrect
from textblob import TextBlob
from profanity import profanity

import contractions
import nltk
nltk.download('all')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet

from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
from sklearn import svm

from sklearn.metrics import f1_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /usr/share/nltk_

In [43]:
main_df=pd.read_csv("../input/trollquestion/train_df.csv")
test_df=pd.read_csv("../input/trollquestion/test_df.csv")
sub_df=pd.read_csv("../input/trollquestion/sample_submission.csv")

In [44]:
main_df_copy=main_df

In [45]:
main_df_copy_x=main_df_copy.drop(axis="columns",labels="target")
main_df_copy_y=main_df_copy["target"].astype(np.float64)

In [46]:
def removeURL(column):
    return column.apply(lambda x: re.sub(r'http\S+', '', x))
def toLetter(column):
    return column.apply(lambda x : re.sub('[^a-zA-Z]',' ',x))

def expandContractions(column):
  contractions_expanded = column.apply(lambda x: [contractions.fix(word) for word in x.split()])
  return [' '.join(map(str, x)) for x in contractions_expanded]

def lowerCase(column):
  return column.apply(lambda x: [word.lower() for word in x])

def tokenize(data_column):
  print(np.shape(data_column))
  return data_column.apply(word_tokenize)
def remove_stopwords(data_column):
  stop_words = set(stopwords.words('english'))
  return data_column.apply(lambda x: [word for word in x if word not in stop_words])
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        
def pos_tags(data_column):
  return data_column.apply(nltk.tag.pos_tag)
                   
def to_wordnet(data_column):
  return data_column.apply(lambda words: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in words])
def word_lemmatizer(data_column):
  wnl = WordNetLemmatizer()
  return data_column.apply(lambda words: [wnl.lemmatize(word, tag) for word, tag in words])

def word_lemmatizer(data_column):
  wnl = WordNetLemmatizer()
  return data_column.apply(lambda words: [wnl.lemmatize(word, tag) for word, tag in words])


In [47]:
def pre_processing(data, df):
  data=removeURL(data)
  data=toLetter(data)
  df['sentences'] = expandContractions(data)
  data=tokenize(df['sentences'])
  df.drop('sentences', axis='columns', inplace=True)
  data = lowerCase(data)
  data = remove_stopwords(data)
  data = pos_tags(data)
  data = to_wordnet(data)
  return word_lemmatizer(data)

In [48]:
main_df_copy_x["data"]=pre_processing(main_df_copy_x["question_text"],main_df_copy_x)

(1000000,)


In [51]:
main_df.head()

,qid,question_text,target
0,dda0b0efc8ba86e81ec4,What are interesting facts about Microsoft his...,0
1,dc708b74a108d0fc0ad9,What are those things which are not gonna happ...,0
2,06a27ec5d82dacd8bfe0,"What should I know to avoid being ""upsold"" whe...",0
3,00cbb6b17e3ceb7c5358,How I add any account with payment bank?,0
4,7c304888973a701585a0,Which Multi level marketing products are actua...,0


In [49]:
main_df_copy_x["data"].head()

0              [interesting, fact, microsoft, history]
1                            [thing, go, happen, ever]
2       [know, avoid, upsold, get, car, brake, change]
3                        [add, account, payment, bank]
4    [multi, level, marketing, product, actually, w...
Name: data, dtype: object

In [52]:
main_df_copy_x['data'] = main_df_copy_x['data'].apply(lambda words : ' '.join([w for w in words]))
main_df_copy_x.head()

,qid,question_text,data
0,dda0b0efc8ba86e81ec4,What are interesting facts about Microsoft his...,interesting fact microsoft history
1,dc708b74a108d0fc0ad9,What are those things which are not gonna happ...,thing go happen ever
2,06a27ec5d82dacd8bfe0,"What should I know to avoid being ""upsold"" whe...",know avoid upsold get car brake change
3,00cbb6b17e3ceb7c5358,How I add any account with payment bank?,add account payment bank
4,7c304888973a701585a0,Which Multi level marketing products are actua...,multi level marketing product actually worth p...


In [53]:
test_df["data"]=pre_processing(test_df["question_text"],test_df)

(306122,)


In [54]:
test_df['data'] = test_df['data'].apply(lambda words : ' '.join([w for w in words]))
test_df.head()

,qid,question_text,data
0,a4f3da3a3df9dd881edd,My period is due on my wedding day. How can I ...,period due wedding day stop pill option
1,9914c62ed3f69684d549,How many numbers higher than a million can be ...,many number high million form digit
2,8138ae48649e37091a91,"How come I feel nothing for my family, but sti...",come feel nothing family still love pet friend...
3,981b4753d17ef14d09f7,"In case of collapse of the Democratic party, w...",case collapse democratic party republican part...
4,452e2c705276ba16b7b7,Who is Émile Naoumoff?,mile naoumoff


In [55]:
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(main_df_copy_x['data'])
train_cv = vectorizer.transform(main_df_copy_x["data"])

In [56]:
test_cv=vectorizer.transform(test_df["data"].to_numpy())

#### calculating the profanity index for the question text

In [57]:
# def prof(column):
#     arr=np.zeros(column.size)
#     count=0
#     for i in column:
#         arr[count]=profanity.contains_profanity(i)
#         count+=1
#     return arr

In [58]:
# profanity_train=prof(main_df["question_text"])
# profanity_test=prof(test_df["question_text"])

In [59]:
# train_cv=scipy.sparse.hstack((train_cv,profanity_train.reshape(-1,1))).tocsr()
# test_cv=scipy.sparse.hstack((test_cv,profanity_test.reshape(-1,1))).tocsr()

In [60]:
model = LogisticRegression(max_iter=2500,class_weight={0:0.15,1:0.85})

In [61]:
model.fit(train_cv,main_df_copy_y)

LogisticRegression(class_weight={0: 0.15, 1: 0.85}, max_iter=2500)

In [62]:
pred_train=model.predict(train_cv)
f1_score(main_df_copy_y,pred_train)

0.7998937724073828

In [63]:
pred_test=model.predict(test_cv)

In [64]:
test_lr_cv_pred=pd.DataFrame(pred_test)
datasets=pd.concat([sub_df['qid'],test_lr_cv_pred],axis=1)
datasets.columns=["qid","target"]

In [65]:
datasets["target"]=datasets['target'].astype(np.int64)

In [66]:
datasets.to_csv("./te_am_prep_0.15_0.85.csv",index=False)